In [1]:
# Import modules
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from us import states
import os
import regex as re
import requests
import json
from bs4 import BeautifulSoup

In [2]:
abbreviation_to_name = {
    # https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States#States.
    "AK": "Alaska",
    "AL": "Alabama",
    "AR": "Arkansas",
    "AZ": "Arizona",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DE": "Delaware",
    "FL": "Florida",
    "GA": "Georgia",
    "HI": "Hawaii",
    "IA": "Iowa",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "MA": "Massachusetts",
    "MD": "Maryland",
    "ME": "Maine",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MO": "Missouri",
    "MS": "Mississippi",
    "MT": "Montana",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "NE": "Nebraska",
    "NH": "New Hampshire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NV": "Nevada",
    "NY": "New York",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PA": "Pennsylvania",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VA": "Virginia",
    "VT": "Vermont",
    "WA": "Washington",
    "WI": "Wisconsin",
    "WV": "West Virginia",
    "WY": "Wyoming",
    # https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States#Federal_district.
    "DC": "District of Columbia",
    # https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States#Inhabited_territories.
    "AS": "American Samoa",
    "GU": "Guam GU",
    "MP": "Northern Mariana Islands",
    "PR": "Puerto Rico PR",
    "VI": "U.S. Virgin Islands",
}

In [3]:
# open fips.csv
fips = pd.read_csv('fips.csv')

fips

,city,county,state,state fips,county fips,place
0,berkeley,alameda,california,6,1,6000
1,san leandro,alameda,california,6,1,68084
2,san francisco,san francisco,california,6,75,67000
3,oakland,alameda,california,6,1,53000
4,minneapolis,hennepin,minnesota,27,53,43000


In [49]:
# open key_rows_with_tracts.csv
krwt = pd.read_excel('key_rows_with_tracts.xlsx')

groups = ['DP02', 'DP03', 'DP05']
# groups = ['DP02']
key = 'ad8851f3bf6aaf76923ec4119b6f714cdfaa87d9'

In [47]:
def centerYear(year):
    if year == 2010:
        return 2010
    if year <= 2006:
        year = 2009
    elif year >= 2020:
        year = 2022
    else: 
        year = year + 2
    return year

In [56]:
# all_data = []
for index, row in krwt.iterrows():
    # find row in fips where city is row[city].lower()
    city = row['city'].lower()
    state = abbreviation_to_name[row['state']].lower()
    fips_row = fips[(fips['city'] == city)]

    county = fips_row['county'].reset_index(drop=True)[0]
    state_code = str(fips_row['state fips'].reset_index(drop=True)[0]).zfill(2)
    county_code = str(fips_row['county fips'].reset_index(drop=True)[0]).zfill(3)

    year = int(row['year'])

    year = centerYear(year)

    if row['geography'] == 'city':
        continue
        place_code = str(fips_row['place'].reset_index(drop=True)[0]).zfill(5)
        
        data_dict = {'year': row['year'],
                                    'city_name': city,
                                  'state_name': state,
                                  'state_code': state_code,
                                  'county_name': county,
                                  'county_code': county_code,
                                  'place_code': place_code}
        for group in groups:        
            url = f"https://api.census.gov/data/{year}/acs/acs5/profile?get=group({group})&for=place:{place_code}&in=state:{state_code}&key={key}"
            try: 
                response = requests.get(url)
                data = json.loads(response.text)
                # append each array of data to each array of all_data
                for i in range(len(data[0])):
                    if re.search(r'\d(?:E|EA)$', data[0][i]) is not None:    
                        data_dict[data[0][i]] = data[1][i]
            except Exception as e:
                print(url)
        all_data.append(data_dict)

    elif city == 'oakland':
        if int(row['year']) != 2012:
            continue 
        print("row " + str(index))
        print(row)
        tracts = row['tracts'].strip('][').split(', ')
        for tract in tracts:
            decremented = False
            tract = tract.zfill(6)
            if tract == '000000':
                continue
            print(tract)
            
            data_dict = {'year': row['year'],
                                        'tract': tract,
                                        'city_name': city,
                                        'state_name': state,
                                        'state_code': state_code,
                                        'county_name': county,
                                        'county_code': county_code
                                        }
            for group in groups:
                dontBreak = True
                while dontBreak:
                    url = f"https://api.census.gov/data/{year}/acs/acs5/profile?get=group({group})&for=tract:{tract}&in=state:{state_code}+county:{county_code}&key={key}"
                    try: 
                        print(url)
                        response = requests.get(url)
                        data = json.loads(response.text)
                        # append each array of data to each array of all_data
                        for i in range(len(data[0])):
                            if re.search(r'\d(?:E|EA)$', data[0][i]) is not None:    
                                data_dict[data[0][i]] = data[1][i]
                        county = fips_row['county'].reset_index(drop=True)[0]
                        county_code = str(fips_row['county fips'].reset_index(drop=True)[0]).zfill(3)
                        dontBreak = False
                        year = centerYear(int(row['year']))
                    except Exception as e:
                        if county_code == '001':
                            county_code = '013'
                            county = 'contra costa'
                        elif county_code == '013':
                            county_code = '037'
                            county = 'los angeles'
                        elif county_code == '037':
                            county_code = '075'
                            county = 'san francisco'
                        elif county_code == '075':
                            year -= 1
                            county = 'alameda'
                            county_code = '001'
                        else:
                            raise Exception("FAIL")
            all_data.append(data_dict)

    

    # https://api.census.gov/data/2010/acs/acs5/profile?get=group(DP05)&for=place:67000&in=state:06&key=d0a5018fadae7c974ffc88620aed6fd71d275fa6

row 90
filename                  Oakland_11062012_CityCouncilDistrict1.csv
city                                                        Oakland
state                                                            CA
geography                                              city council
district                                                        1.0
office                                                 City council
year                                                           2012
geo_filename                          oakland_city_council_2003.csv
tract_filename                              oakland_tracts_2010.csv
tracts            [401300, 404501, 400300, 400600, 401000, 40140...
percents          [8.108544361627887e-05, 0.013812172951728242, ...
Name: 90, dtype: object
401300
https://api.census.gov/data/2014/acs/acs5/profile?get=group(DP02)&for=tract:401300&in=state:06+county:001&key=d0a5018fadae7c974ffc88620aed6fd71d275fa6
https://api.census.gov/data/2014/acs/acs5/profile?get=group(DP0

KeyboardInterrupt: 

In [54]:
columns = set().union(*(d.keys() for d in all_data))

filled_dicts = [{key: d.get(key) for key in columns} for d in all_data]

df = pd.DataFrame(filled_dicts)

In [55]:
# delete all rows from df where state, year, county, tract, and place values are the same
df.drop_duplicates(subset=['tract', 'year', 'state_code', 'county_code'], keep='last', inplace=True)

# order so that place, county, state, year are first
df = df[['tract', 'city_name', 'state_name', 'state_code', 'county_name', 'county_code', 'year'] + [col for col in df.columns if col not in ['city_name', 'place_code', 'county_name', 'county_code', 'state_name', 'state_code', 'year']]]

df.to_csv(r'acs data\oakland_tract_data.csv', index=False)

In [31]:
df

,tract,city_name,state_name,state_code,county_name,county_code,year,DP03_0040E,DP03_0116E,DP02_0134E,...,DP05_0008E,DP05_0091EA,DP03_0130E,DP03_0035EA,DP03_0042E,DP03_0004EA,DP03_0001EA,DP02_0090EA,DP03_0025E,DP02_0115E
0,432501,san leandro,california,06,alameda,001,2010,80,271,87,...,233,None,-888888888,None,372,None,None,None,22.3,628
1,430400,san leandro,california,06,alameda,001,2010,76,140,135,...,32,None,-888888888,None,224,None,None,None,30.2,87
2,433104,san leandro,california,06,alameda,001,2010,159,236,353,...,155,None,-888888888,None,253,None,None,None,27.9,326
5,433000,san leandro,california,06,alameda,001,2010,103,215,199,...,209,None,-888888888,None,297,None,None,None,26.5,363
8,433103,san leandro,california,06,alameda,001,2010,102,272,67,...,285,None,-888888888,None,380,None,None,None,30.7,1222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,433400,san leandro,california,06,alameda,001,2020,259,325,0,...,299,None,-888888888,None,725,None,None,None,34.1,1204
133,433500,san leandro,california,06,alameda,001,2020,221,252,0,...,156,None,-888888888,None,517,None,None,None,32.1,1348
134,435800,san leandro,california,06,alameda,001,2020,134,349,5,...,438,None,-888888888,None,589,None,None,None,32.5,1504
135,435900,san leandro,california,06,alameda,001,2020,259,467,0,...,273,None,-888888888,None,550,None,None,None,32.5,1125
